# Code to analyse BCD output before (first-fit) and after adjustig to max stable gain

22 July 2023 Guido Cattani

Is it worth to try to set gain to maximal stable values?

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon as wilcoxon
from scipy.stats import shapiro as shapiro
from scipy.stats import ttest_rel as ttest_rel

In [2]:
# read output force levels measured with a 65 dB ISDS input signal
def read_BCD_output_65():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Implant/Primary_data/output_BCD_65dB.csv'
    p_in = Path(f_in)   
    df = pd.read_csv(p_in)
    df.set_index('Study_ID', inplace=True)
    df = df.fillna(pd.NA)
    return df

In [3]:
# read output force levels measured with a 65 dB ISDS input signal
def read_maximal_output_65():
    f_in = '/media/guido/LACIE/Cingle_Guido/Master/Implant/Primary_data/max_stable_output_1500_65dB.csv'
    p_in = Path(f_in)   
    df = pd.read_csv(p_in)
    df.set_index('Study_ID', inplace=True)
    df = df.fillna(pd.NA)
    return df

In [4]:
def diff_output():
    bcd_fit = read_BCD_output_65()
    bcd_max = read_maximal_output_65()
    diff = bcd_max - bcd_fit
    return diff

In [5]:
def descriptive_stat(df):
    # calculate quantiles
    quantiles = df.quantile([0.1, 0.5, 0.9]).round(1)
    quantiles.index = ['P10', 'P50', 'P90']
 
    # calculate mean and standard deviation
    mean_values = (pd.DataFrame({'Mean': df.mean().round(1)})).T
    std_values = (pd.DataFrame({'St. dev.': df.std().round(1)})).T

    res = pd.concat([quantiles, mean_values, std_values])
    return res.round(1)

In [6]:
def shapiro_test_norm(df):
    ''' Perform the Shapiro-Wilk test for normality. 
    The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution.'''
    
    shapiro_result = df.apply(lambda x: shapiro(x) if len(x) >= 3 else (float('nan'), float('nan')))
    
    shapiro_stats = shapiro_result.apply(lambda x: round(x[0], 3))
    p_values = shapiro_result.apply(lambda x: round(x[1], 3))
    is_normal = p_values >= 0.05
    
    shapiro_test = pd.DataFrame({
        'Shapiro test statistic': shapiro_stats,
        'p-value': p_values,
        'normally distributed': is_normal
    }).transpose()
    
    return shapiro_test

In [7]:
def normal_couple(df1, df2):
    # normality of 2 df
    normality1 = shapiro_test_norm(df1).loc['normally distributed']
    normality2 = shapiro_test_norm(df2).loc['normally distributed']
    both_normal = normality1 & normality2
    normality = pd.DataFrame({
        'first-fit output is normal distributed': normality1,
        'max output is normal distributed': normality2,
        'max & first-fit are normal distributed': both_normal
    }).transpose()
    normality = normality.astype(bool)
    return normality

In [8]:
def t_test_rel(df1, df2):
    ''' Calculate the t-test on TWO RELATED samples of scores, a and b. 
    This is a test for the null hypothesis that two related or repeated samples have identical average (expected) values.'''

    ttest_result = df1.apply(lambda x: ttest_rel(x, df2[x.name]) if len(x) >= 3 else (float('nan'), float('nan')))

    ttest_stats = ttest_result.apply(lambda x: round(x[0], 3))
    p_values = ttest_result.apply(lambda x: round(x[1], 3))
    #dfr = ttest_result.apply(lambda x: x[2])

    ttest_test = pd.DataFrame({
        'ttest test statistic': ttest_stats,
        'p-value ttest': p_values,
    }).transpose()

    return ttest_test

In [9]:
def wilcoxon_signed_rank(df):
    ''' Compare output of BAHA5P at 65 dB, first fit versus maximal stable
    The Wilcoxon signed-rank test tests the null hypothesis that 
    two related paired samples come from the same distribution. 
    In particular, it tests whether the distribution of the differences x - y is symmetric 
    about zero. It is a non-parametric version of the paired T-test.'''
    
    wilcoxon_result = df.apply(lambda x: wilcoxon(x) if len(x) >= 3 else (float('nan'), float('nan')))
    
    wilcoxon_stats = wilcoxon_result.apply(lambda x: round(x[0], 3))
    p_values = wilcoxon_result.apply(lambda x: round(x[1], 3))
    
    wilcoxon_test = pd.DataFrame({
        'wilcoxon test statistic': wilcoxon_stats,
        'p-value wilcoxon': p_values,
    }).transpose()
    
    return wilcoxon_test

In [10]:
diff = diff_output()

In [11]:
len(diff)

20

In [12]:
diff

,f_250_Hz,f_315_Hz,f_400_Hz,f_500_Hz,f_630_Hz,f_800_Hz,f_1000_Hz,f_1250_Hz,f_1600_Hz,f_2000_Hz,f_2500_Hz,f_3150_Hz,f_4000_Hz,f_5000_Hz,f_6300_Hz,f_8000_Hz
Study_ID,,,,,,,,,,,,,,,,
34,3.9,3.9,10.0,10.2,9.4,9.1,7.3,3.3,3.2,4.0,1.0,0.2,4.0,8.0,8.7,5.8
35,4.6,4.9,4.6,5.2,5.1,4.5,4.6,4.8,4.4,4.7,9.7,11.9,6.1,2.0,0.1,0.0
43,3.1,1.2,2.2,2.7,2.9,3.3,4.7,7.3,8.7,8.4,6.3,4.8,4.1,4.7,6.1,6.4
47,9.8,9.3,9.4,9.5,9.8,10.3,11.0,12.8,15.6,18.9,20.9,17.0,11.1,8.1,6.4,6.0
50,3.1,2.7,3.9,4.1,4.5,5.3,5.8,5.7,8.7,12.8,12.6,8.1,4.4,5.1,5.6,5.9
53,3.6,3.8,2.5,2.9,3.0,2.6,3.0,6.4,11.8,16.9,16.0,11.9,6.3,5.1,5.9,6.4
54,8.2,12.6,11.0,4.4,21.4,19.3,15.5,12.5,13.6,15.3,12.2,7.6,5.0,6.0,7.2,7.3
56,5.6,6.4,7.4,7.0,6.8,6.6,6.2,5.3,7.2,10.9,10.3,6.2,1.5,0.2,1.4,2.3
59,6.0,5.8,4.7,5.6,6.3,6.7,6.9,7.5,10.2,14.6,17.0,12.6,4.8,1.9,2.5,3.0


In [13]:
des_stat = descriptive_stat(diff)

In [14]:
first_fit_output = read_BCD_output_65()
max_output = read_maximal_output_65()
normal2check = normal_couple(first_fit_output, max_output)

In [15]:
ttr = t_test_rel(max_output, first_fit_output)
ttr = ttr * normal2check.loc['max & first-fit are normal distributed'].replace(False,pd.NA)

In [16]:
wsr = wilcoxon_signed_rank(diff)
wsr = wsr * (~normal2check.loc['max & first-fit are normal distributed'].replace(True,pd.NA)) * -1

/home/guido/anaconda3/lib/python3.9/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [17]:
results = pd.concat([des_stat, normal2check, ttr, wsr])
results.drop(index = ['first-fit output is normal distributed', 'max output is normal distributed'], inplace = True)
results

,f_250_Hz,f_315_Hz,f_400_Hz,f_500_Hz,f_630_Hz,f_800_Hz,f_1000_Hz,f_1250_Hz,f_1600_Hz,f_2000_Hz,f_2500_Hz,f_3150_Hz,f_4000_Hz,f_5000_Hz,f_6300_Hz,f_8000_Hz
P10,0.2,1.2,0.5,-1.2,-1.2,-0.8,0.5,1.4,3.0,4.7,6.3,4.2,1.8,1.7,1.3,1.4
P50,3.6,4.1,4.4,3.9,4.0,4.0,5.7,7.4,10.8,11.7,11.3,8.4,4.6,4.2,5.4,5.8
P90,6.5,9.5,9.5,8.7,9.4,9.4,10.2,12.0,13.6,17.1,17.1,12.9,9.2,8.0,7.4,6.9
Mean,3.9,4.8,4.5,3.9,4.9,5.1,5.6,6.8,9.0,11.4,11.7,8.8,4.9,4.1,4.5,4.7
St. dev.,2.6,3.5,3.7,3.7,5.3,4.8,4.0,4.0,4.3,4.8,4.8,4.1,2.8,2.5,2.7,2.6
max & first-fit are normal distributed,True,True,True,True,False,False,False,True,True,True,True,True,True,False,True,True
ttest test statistic,6.661,6.184,5.423,4.769,<NA>,<NA>,<NA>,7.552,9.301,10.675,10.965,9.532,7.855,<NA>,7.34,8.084
p-value ttest,0.0,0.0,0.0,0.0,<NA>,<NA>,<NA>,0.0,0.0,0.0,0.0,0.0,0.0,<NA>,0.0,0.0
wilcoxon test statistic,<NA>,<NA>,<NA>,<NA>,11.0,6.0,2.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2.0,<NA>,<NA>
p-value wilcoxon,<NA>,<NA>,<NA>,<NA>,0.0,0.0,0.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,<NA>,<NA>


In [18]:
# write to csv file
results.to_csv("/media/guido/LACIE/Cingle_Guido/Master/Implant/Analysis_Results/analysis_diff_max_vs_firstfit.csv")  